# Libraries

In [34]:
import numpy as np
import pandas as pd
import operator
from copy import deepcopy
from math import log

# Load Data

In [35]:
filepath = 'spliceATrainKIS.dat'
data = []

In [36]:
with open(filepath) as fp:
    line = fp.readline()
    cnt = 1
    while line:
        data.append(line.strip())
        line = fp.readline()
        cnt += 1

In [37]:
np.shape(data)

(11577,)

In [38]:
cutNr = int(data[0])

In [39]:
cutNr

68

In [40]:
data = data[1:]

In [41]:
data = np.array(data)

In [42]:
data = np.reshape(a=data, newshape=(int(11576/2), 2))

In [43]:
df = pd.DataFrame(data, columns=['y', "seq"])

In [44]:
df.y = pd.to_numeric(df.y)
classes = list(df.y)

In [45]:
attributes = set("".join([i for i in df.seq]))

In [46]:
sequences = np.array([list(i) for i in df.seq])

In [47]:
attributes


{'A', 'C', 'G', 'N', 'S', 'T'}

# Algorytm 

zbiór klas 'c'- czy jest prawdziwy czy nie (1 & 0)

zbiór obiektów 's' - zbiór kolejnych przykładów

zbiór atrybutów poza klasą 'r'- zbiór unikalnych liter

## częstość i-tej klasy

In [48]:
#znalezc ceche, ktora ma najwieksza czestosc
p = [{c: 0 for c in attributes} for i in range(sequences.shape[1])]
n = deepcopy(p)
pAn = deepcopy(p)
for ridx, row in enumerate(sequences):
    for cidx, column in enumerate(row):
        pAn[cidx][column] += 1
        if df.y[ridx] == 1:
            p[cidx][column] += 1
        else:
            n[cidx][column] += 1
        

mam zliczoną ilość wystąpień klejnych klas w danych numerach algorytmu

In [49]:
p[:5]

[{'G': 308, 'N': 2, 'T': 290, 'C': 254, 'A': 262, 'S': 0},
 {'G': 319, 'N': 3, 'T': 263, 'C': 266, 'A': 265, 'S': 0},
 {'G': 292, 'N': 1, 'T': 281, 'C': 287, 'A': 255, 'S': 0},
 {'G': 337, 'N': 1, 'T': 268, 'C': 250, 'A': 260, 'S': 0},
 {'G': 313, 'N': 1, 'T': 258, 'C': 289, 'A': 255, 'S': 0}]

In [50]:
n[:5]

[{'G': 1361, 'N': 1, 'T': 1143, 'C': 1160, 'A': 1007, 'S': 0},
 {'G': 1293, 'N': 0, 'T': 1192, 'C': 1135, 'A': 1052, 'S': 0},
 {'G': 1270, 'N': 0, 'T': 1178, 'C': 1185, 'A': 1039, 'S': 0},
 {'G': 1255, 'N': 1, 'T': 1197, 'C': 1182, 'A': 1037, 'S': 0},
 {'G': 1276, 'N': 1, 'T': 1185, 'C': 1164, 'A': 1046, 'S': 0}]

In [51]:
pAn[:5]

[{'G': 1669, 'N': 3, 'T': 1433, 'C': 1414, 'A': 1269, 'S': 0},
 {'G': 1612, 'N': 3, 'T': 1455, 'C': 1401, 'A': 1317, 'S': 0},
 {'G': 1562, 'N': 1, 'T': 1459, 'C': 1472, 'A': 1294, 'S': 0},
 {'G': 1592, 'N': 2, 'T': 1465, 'C': 1432, 'A': 1297, 'S': 0},
 {'G': 1589, 'N': 2, 'T': 1443, 'C': 1453, 'A': 1301, 'S': 0}]

In [52]:
def id3(classes, attributes, sequences):
    """
    c- list of classes
    r- list of atributes
    s- list of objects
    """
    if not sequences:
        return -1
    if classes.count(classes[0]) == len(x):
        return classes[0]
    if not attributes:
        stats = {label: list(classes).count(label)  for label in np.unique(classes)}
        max(stats.items(), key=operator.itemgetter(1))[0]
    #atrybut maksymalizujący InfGain(D, S)
    d = 0

## atrybut maksimalizujący infGain

### entropia zbioru

In [53]:
testD = {'N': 1, 'T': 1143, 'S': 0, 'A': 1007, 'C': 1160, 'G': 1361}
for i in testD:
    print(i)

N
T
S
A
C
G


In [54]:
def calculate_frequency(pAn):
    """
    p = list of dictionaries caunting positive
    n - list of dictionaries counting negative
    y - output (0, 1)
    """
    totalInRow = [sum(row.values()) for row in pAn]
    frequencies = deepcopy(pAn)
    for row, rowSum in zip(frequencies, totalInRow):
        for key in row:
            row[key] = row[key]/rowSum
    return frequencies
frequencies = calculate_frequency(pAn)

In [55]:
def calculate_entropyLabel(labels):
    p = 0
    n = 0
    for ridx, row in enumerate(labels):
        if row == 1:
            p += 1
        else:
            n += 1
    pf = p/len(labels)
    nf = n/len(labels)
    entropy = -pf*log(pf, 2)-nf*log(nf, 2)
    return entropy

In [56]:
entropyL = calculate_entropyLabel(classes)

policzyc entropy dla outlook z przykładu zeby sprawdzic dzialanie

In [57]:
def singleEntropy(f):
    return -f*log(f, len(frequencies))
def entropy(e1, e2, *rest):
    print(rest)
    args = np.concatenate(([e1, e2], rest)).astype(int) 
    frequences = [arg/sum(args) for arg in args]
    return sum([singleEntropy(f) for f in frequencies])
    

In [58]:
entropy(1, 2)

TypeError: entropy() missing 1 required positional argument: 'e2'

In [ ]:
eW = entropy(6, 2)
eW
eS = entropy(3, 3)
eS

In [ ]:
def informationGain(eClasses, a0, a1, entropy0, entropy1):
    f0 = a0/(a0+a1)
    f1 = a1/(a0+a1)   
    ig = eClasses - sum([f0*entropy0, f1*entropy1])
    return ig

In [ ]:
informationGain(0.94, 8,  6, eW, eS)

In [ ]:
lista = [1, 2, 3, 4]
[arg/ for idx, arg in enumerate(lista)]